## Label Refinement
This notebook is used for refining labels in manually segmented images (for semantic segmentation). Manual annotation usually causes changes in pixel values near object boundaries. This notebook caters to refine that.

In [1]:
import numpy as np
import cv2
import json
import os

In [2]:
# Specify the dataset directories and the classes file path
data_dir = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSeg'
json_path = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSegClasses.json'

# Save directory
save_dir = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSegRefined'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
files = {x: [os.path.join(data_dir, x, 'groundtruth', f) for f in os.listdir(os.path.join(data_dir, x, 'groundtruth')) 
         if (f.endswith('.jpg') or f.endswith('.png'))]
         for x in ['train', 'test', 'trainval']}

In [4]:
def disentangleKey(key):
    '''
        Disentangles the key for class and labels obtained from the
        JSON file
        Returns a python dictionary of the form:
            {Class Id: RGB Color Code as numpy array}
    '''
    dKey = {}
    for i in range(len(key)):
        class_id = int(key[i]['id'])
        c = key[i]['color']
        c = c.split(',')
        c0 = int(c[0][1:])
        c1 = int(c[1])
        c2 = int(c[2][:-1])
        color_array = np.asarray([c0,c1,c2])
        dKey[class_id] = color_array

    return dKey

In [5]:
# Get the classes RGB key
classes = json.load(open(json_path))['classes']
key = disentangleKey(classes)

In [6]:
# Please run only once, otherwise restart kernel and then run again
for k in range(len(key)):
    rgb = key[k]
    rgb = np.expand_dims(rgb, 0)
    if 'keyMat' in locals():
        keyMat = np.concatenate((keyMat, rgb), axis=0)
    else:
        keyMat = rgb

In [7]:
# Iterate over all images to smooth them
x = ['train', 'test', 'trainval']
for i in range(len(files)):
    folder = files[x[i]]
    save_sub_dir = os.path.join(save_dir, x[i], 'groundtruth')
    if not os.path.exists(save_sub_dir):
        os.makedirs(save_sub_dir)
        
    for j in range(len(folder)):
        img = cv2.imread(folder[j])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        refined = np.zeros_like(img)
        
        # Iterate over all image pixels
        # TODO: Vectorize this
        for h in range(img.shape[0]):
            for w in range(img.shape[1]):
                
                label = np.argmin(np.linalg.norm(np.subtract(img[h,w,:], keyMat), axis = 1))
                rgb = key[label]
                refined[h,w,:] = rgb
        
        # Apply median filtering to remove the salt pepper noise produced at image boundaries
        refined = cv2.medianBlur(refined, 5)
        
        # Save the image
        refined = cv2.cvtColor(refined, cv2.COLOR_BGR2RGB)
        file_name = folder[j].split('/')[-1].split('.')[0] + '.png'
        save_path = os.path.join(save_sub_dir, file_name)
        cv2.imwrite(save_path, refined)
        print('Image: [%d]/[%d]: Folder: [%d]/[%d]' % (j+1, len(folder), i+1, len(files)))

Image: [1]/[192]: Folder: [1]/[3]
Image: [2]/[192]: Folder: [1]/[3]
Image: [3]/[192]: Folder: [1]/[3]
Image: [4]/[192]: Folder: [1]/[3]
Image: [5]/[192]: Folder: [1]/[3]
Image: [6]/[192]: Folder: [1]/[3]
Image: [7]/[192]: Folder: [1]/[3]
Image: [8]/[192]: Folder: [1]/[3]
Image: [9]/[192]: Folder: [1]/[3]
Image: [10]/[192]: Folder: [1]/[3]
Image: [11]/[192]: Folder: [1]/[3]
Image: [12]/[192]: Folder: [1]/[3]
Image: [13]/[192]: Folder: [1]/[3]
Image: [14]/[192]: Folder: [1]/[3]
Image: [15]/[192]: Folder: [1]/[3]
Image: [16]/[192]: Folder: [1]/[3]
Image: [17]/[192]: Folder: [1]/[3]
Image: [18]/[192]: Folder: [1]/[3]
Image: [19]/[192]: Folder: [1]/[3]
Image: [20]/[192]: Folder: [1]/[3]
Image: [21]/[192]: Folder: [1]/[3]
Image: [22]/[192]: Folder: [1]/[3]
Image: [23]/[192]: Folder: [1]/[3]
Image: [24]/[192]: Folder: [1]/[3]
Image: [25]/[192]: Folder: [1]/[3]
Image: [26]/[192]: Folder: [1]/[3]
Image: [27]/[192]: Folder: [1]/[3]
Image: [28]/[192]: Folder: [1]/[3]
Image: [29]/[192]: Folder: [1

Image: [41]/[115]: Folder: [2]/[3]
Image: [42]/[115]: Folder: [2]/[3]
Image: [43]/[115]: Folder: [2]/[3]
Image: [44]/[115]: Folder: [2]/[3]
Image: [45]/[115]: Folder: [2]/[3]
Image: [46]/[115]: Folder: [2]/[3]
Image: [47]/[115]: Folder: [2]/[3]
Image: [48]/[115]: Folder: [2]/[3]
Image: [49]/[115]: Folder: [2]/[3]
Image: [50]/[115]: Folder: [2]/[3]
Image: [51]/[115]: Folder: [2]/[3]
Image: [52]/[115]: Folder: [2]/[3]
Image: [53]/[115]: Folder: [2]/[3]
Image: [54]/[115]: Folder: [2]/[3]
Image: [55]/[115]: Folder: [2]/[3]
Image: [56]/[115]: Folder: [2]/[3]
Image: [57]/[115]: Folder: [2]/[3]
Image: [58]/[115]: Folder: [2]/[3]
Image: [59]/[115]: Folder: [2]/[3]
Image: [60]/[115]: Folder: [2]/[3]
Image: [61]/[115]: Folder: [2]/[3]
Image: [62]/[115]: Folder: [2]/[3]
Image: [63]/[115]: Folder: [2]/[3]
Image: [64]/[115]: Folder: [2]/[3]
Image: [65]/[115]: Folder: [2]/[3]
Image: [66]/[115]: Folder: [2]/[3]
Image: [67]/[115]: Folder: [2]/[3]
Image: [68]/[115]: Folder: [2]/[3]
Image: [69]/[115]: F

Image: [159]/[307]: Folder: [3]/[3]
Image: [160]/[307]: Folder: [3]/[3]
Image: [161]/[307]: Folder: [3]/[3]
Image: [162]/[307]: Folder: [3]/[3]
Image: [163]/[307]: Folder: [3]/[3]
Image: [164]/[307]: Folder: [3]/[3]
Image: [165]/[307]: Folder: [3]/[3]
Image: [166]/[307]: Folder: [3]/[3]
Image: [167]/[307]: Folder: [3]/[3]
Image: [168]/[307]: Folder: [3]/[3]
Image: [169]/[307]: Folder: [3]/[3]
Image: [170]/[307]: Folder: [3]/[3]
Image: [171]/[307]: Folder: [3]/[3]
Image: [172]/[307]: Folder: [3]/[3]
Image: [173]/[307]: Folder: [3]/[3]
Image: [174]/[307]: Folder: [3]/[3]
Image: [175]/[307]: Folder: [3]/[3]
Image: [176]/[307]: Folder: [3]/[3]
Image: [177]/[307]: Folder: [3]/[3]
Image: [178]/[307]: Folder: [3]/[3]
Image: [179]/[307]: Folder: [3]/[3]
Image: [180]/[307]: Folder: [3]/[3]
Image: [181]/[307]: Folder: [3]/[3]
Image: [182]/[307]: Folder: [3]/[3]
Image: [183]/[307]: Folder: [3]/[3]
Image: [184]/[307]: Folder: [3]/[3]
Image: [185]/[307]: Folder: [3]/[3]
Image: [186]/[307]: Folder: 